<a href="https://colab.research.google.com/github/hxtruong/fb-developer-circles/blob/master/week_6/Assignment_3_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns. set_style("whitegrid")

# To avoid Warning message inbetween ...
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df = pd.read_csv('https://raw.githubusercontent.com/hxtruong/fb-developer-circles/master/week_6/labeledTrainData.tsv', sep='\t', encoding='latin-1')
df.sample(10)

,id,sentiment,review
2800,6617_1,0,Truly shows that hype is not everything. Shows...
15720,2524_4,0,Title: Robot Jox (1990) <br /><br />Director: ...
14771,5718_7,1,"I wanted to see Sarah Buffy on the big screen,..."
10451,2155_3,0,"Goodnight, Mister Tom begins in an impossibly ..."
22618,10475_8,1,"A friend of mine recommended this movie, citin..."
22570,3000_8,1,Michael Curtiz directed this 1930 very-stylish...
19597,3128_9,1,Corny! I love it! Corny - just as the TV show ...
5757,7611_9,1,"\I have looked into the eye of this island, an..."
13020,3410_9,1,I discovered this film after reading the book ...
11247,6046_7,1,I was pleasantly surprised by the film. Let's ...


In [23]:
df.describe()

,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


## 2. Preprocessing

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [0]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)

    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    text = re.sub('(?::|;|=)(?:-)?(?:\)|\(|d|p)','', text.lower())

    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [0]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [0]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split

X = df['review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=99)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [35]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = stop,
                        tokenizer=tokenizer_porter,
                        preprocessor = preprocessor)

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7ff613fe76a8>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7ff613fe7c80>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

## 4. Evaluate Model

In [36]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))


accuracy: 0.8869333333333334
confusion matrix:
 [[3226  481]
 [ 367 3426]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.88      3707
           1       0.88      0.90      0.89      3793

    accuracy                           0.89      7500
   macro avg       0.89      0.89      0.89      7500
weighted avg       0.89      0.89      0.89      7500



## 5. Export Model 

In [0]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open(os.path.join('./', 'logisticRegression.pkl'), 'wb'), protocol=4)